### PINECONE Vector Store

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
embeddings_google = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings_google

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x16b1972d0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [7]:
len(embeddings_google.embed_query("Hello AI"))

768

In [12]:
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

In [13]:
index_name = "agenticindex"
if not pc.has_index(index_name):
    print(f"Creating index {index_name}...")
    pc.create_index(
        name=index_name, 
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [16]:
index = pc.Index(index_name)
index

In [17]:
from langchain_pinecone import PineconeVectorStore
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings_google
)

In [18]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [19]:
from uuid import uuid4
# universal identification number
uuids = [str(uuid4()) for _ in range(len(documents))]

vectorstore.add_documents(documents=documents, ids=uuids)

['9638d330-6f84-4d6b-9694-4fc14cd44e01',
 '90f3fd49-0993-4242-8079-b3fd8aa5cd75',
 'f9681b0a-10cb-432a-8d1e-cfa1a06b7069',
 'e50c8d38-8744-463a-b501-037d8f8d02ad',
 '537d5779-6a91-45bd-a860-1af75baf65c4',
 'df34d2f2-e45d-4810-9b2a-e607919b35e3',
 '5afa06fa-45b1-4846-98ba-ac55b4455799',
 '6458414a-ebdc-4d98-b209-2f042e51ad0b',
 'cd9dbab2-c304-4b96-954a-7345488a0056',
 '0a411da7-d6ef-45d7-aaf5-7b061c5305f9']

In [20]:
results = vectorstore.similarity_search("What langchain provided to us?", k=6)
results

[Document(id='325efa79-9c7b-4822-a8e6-6d6b3fc49561', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='83ea3ae0-6289-4766-be5e-53b1880403e7', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='103abf09-263a-4fc4-8333-1984ec09c940', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(id='f2c06e55-e841-45e1-854c-694f1d68c9a3', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='a5527bc2-eeea-4246-b541-eb62bb6c2f70', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='60e32167-46e5-4760-85cb-b78aaa93eec8', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

In [21]:
results = vectorstore.similarity_search("What langchain provided to us?", filter={"source": "tweet"})
results

[Document(id='325efa79-9c7b-4822-a8e6-6d6b3fc49561', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='83ea3ae0-6289-4766-be5e-53b1880403e7', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='103abf09-263a-4fc4-8333-1984ec09c940', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(id='f2c06e55-e841-45e1-854c-694f1d68c9a3', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again.")]

### Pinecone as Retriever

In [22]:
retriever = vectorstore.as_retriever(search_type = "similarity_score_threshold", search_kwargs={"k": 3, "score_threshold": 0.8})
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x100bbbc10>, search_type='similarity_score_threshold', search_kwargs={'k': 3, 'score_threshold': 0.8})

In [23]:
retriever.invoke("langchain")

[Document(id='f9681b0a-10cb-432a-8d1e-cfa1a06b7069', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='325efa79-9c7b-4822-a8e6-6d6b3fc49561', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='6458414a-ebdc-4d98-b209-2f042e51ad0b', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]